In [15]:
#!usr/bin/env python3
#! -*- coding: utf-8 -*-
#xiaobaodas version_alpha_03 2017-03-17 修补SQL和debug传参可能导致的BUG，添加fillna参数
#Ex-Ch:V_a01->02:优化import方式，修补Visitor表的BUG，添加‘查看详情’、‘同款货源’变量，添加path参数，添加debug=9的CSV导出功能，加入运行时间反馈
def xiaobaods_a02(date="",category="牛仔裤",length=7,SQL="xiaobaods",table="bc_attribute_granularity_sales",variable="热销排名",fillna="",debug=0,path=""):
    import time
    time_s = time.time()
    import pandas as pd
    import datetime
    import pymysql
    from dateutil.parser import parse
    SQL_msg = {"local":{"host":"127.0.0.1","port":3306,"user":"root","charset":"utf8","passwd":"","db":"baoersqlbasic"},
               "xiaobaods":{"host":"101.201.237.58","port":3306,"user":"program_default","charset":"utf8","passwd":"KQPp5wwZJG33fwFs","db":"baoersqlbasic"}}
    latest_date=datetime.datetime.today().date()-datetime.timedelta(1)
    if category not in ["牛仔裤","打底裤","休闲裤"]:
        category="牛仔裤"
    if length>14 or length<3:
        length=7
    if SQL not in SQL_msg:
        SQL=SQL_msg[0]
    if table not in ["bc_attribute_granularity_sales","bc_attribute_granularity_visitor"]:
        table="bc_attribute_granularity_sales"
    if date=="":
        date = latest_date
    else:
        date=parse(date).date()  # 修改日期格式
    if table=="bc_attribute_granularity_sales":
        sql_select_f = "SELECT CT.`主图缩略图`,CT.`热销排名`,CT.`商品信息`,CT.`所属店铺`,CT.`支付子订单数`,CT.`交易增长幅度`,CT.`支付转化率指数`,CT.`宝贝链接`,CT.`店铺链接`,CT.`查看详情`,CT.`同款货源`"
        if variable not in ["热销排名","支付子订单数","交易增长幅度","支付转化率指数"]:
            variable="热销排名"
    elif table=="bc_attribute_granularity_visitor":
        sql_select_f = "SELECT CT.`主图缩略图`,CT.`热销排名`,CT.`商品信息`,CT.`所属店铺`,CT.`流量指数`,CT.`搜索人气`,CT.`支付子订单数`,CT.`宝贝链接`,CT.`店铺链接`,CT.`查看详情`,CT.`同款货源`"
        if variable not in ["热销排名","流量指数","搜索人气","支付子订单数"]:
            variable="热销排名"
    # Try to connect with the mysql and back a date which minimum.
    try:
        conn = pymysql.connect(host=SQL_msg[SQL]["host"], port=SQL_msg[SQL]["port"], user=SQL_msg[SQL]["user"], passwd=SQL_msg[SQL]["passwd"], charset=SQL_msg[SQL]["charset"], db=SQL_msg[SQL]["db"])
        cursor = conn.cursor()
        cursor.execute("SELECT min(`日期`),max(`日期`) from "+table+" where `类目`='"+category+"';")
        date_limit = cursor.fetchall()
        date_floor = date_limit[0][0]
        date_ceiling = date_limit[0][1] 
        cursor.close()
        conn.close()
    except Exception as e:
        return e
    if date > latest_date:
        date = latest_date
    if date > date_ceiling:
        date = date_ceiling
    if date < date_floor + datetime.timedelta(length-1):
        date = date_floor + datetime.timedelta(length-1)
    # Main program.
    sql_select_m=""
    for i in range(length):
        sql_select_m += ",MAX(CASE ST.日期 WHEN "+str(date - datetime.timedelta(length-i-1)).replace("-","")+" THEN ST."+variable+" ELSE NULL END) AS `日期："+str(date - datetime.timedelta(length-i-1)).replace("-","")+"` "
    sql_select_e="FROM "+table+" AS CT LEFT JOIN "+table+" AS ST ON SUBSTRING(CT.`宝贝链接` ,- 12) = SUBSTRING(ST.`宝贝链接` ,- 12) WHERE CT.`日期` = "+str(date).replace("-","")+" AND CT.类目 = '"+category+"' AND ST.日期 >= "+str(date - datetime.timedelta(length)).replace("-","")+" AND ST.类目 = '"+category+"' GROUP BY CT.`"+variable+"` ORDER BY CT.`热销排名`;"
    # read msg from Mysql
    conn = pymysql.connect(host=SQL_msg[SQL]["host"], port=SQL_msg[SQL]["port"], user=SQL_msg[SQL]["user"], passwd=SQL_msg[SQL]["passwd"], charset=SQL_msg[SQL]["charset"], db=SQL_msg[SQL]["db"])
    df = pd.io.sql.read_sql_query(sql_select_f+sql_select_m+sql_select_e,conn)
    conn.close()
    df = df.fillna(fillna)
    if debug not in [1,2,8,9]:
        print(df.to_json(orient="index"))
    elif debug== 8:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        return df
    elif debug== 2:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        print("- date：%r \n- category：%r \n- length：%r \n- SQL：%r \n- table: %r \n- variable：%r \n- debug：%r \n- path: %r"%(str(date),category,str(length),SQL,table,variable,debug,path))
    elif debug== 1:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        print( "  SQL_choice: %r \n- category: %r \n- length: %r \n- date: %r \n- SQL: %r"%(SQL,category,str(length),str(date),sql_select_f+sql_select_m+sql_select_e))
    elif debug==9:
        import os
        print ("- Running time：%.4f s"%(time.time()-time_s))
        path_default=os.path.join(os.path.expanduser("~"), 'Desktop')
        if not os.path.isdir(path):
            path = path_default
        csv_filename="【数据组】["+table.split("_")[-1]+"_"+category+"_Top500"+"]"+variable+"_"+datetime.datetime.strftime(date,"%m%d")+"-"+str(length)+"_"+SQL+".csv"
        try:
            df.to_csv(path+"\\"+csv_filename)
            print("> 输出CSV文件：",path,",",csv_filename)
        except Exception as e:
            print("> 输出CSV文件失败，错误原因：",e)
    if __name__ == "__main__":
        try:
            import sys
            argv = sys.argv[1]
            argv = eval(argv)
        except:
            argv ={}

In [16]:
df1 = xiaobaods_a02(table="bc_attribute_granularity_visito",category='打底裤',length=14,debug=8)

- Running time：3.0078 s


In [8]:
df1

,主图缩略图,热销排名,商品信息,所属店铺,支付子订单数,交易增长幅度,支付转化率指数,宝贝链接,店铺链接,查看详情,...,日期：20170307,日期：20170308,日期：20170309,日期：20170310,日期：20170311,日期：20170312,日期：20170313,日期：20170314,日期：20170315,日期：20170316
0,//img.alicdn.com/bao/uploaded/i3/TB1DV07MpXXXX...,1,南极人春季小脚九分显瘦黑2017新款外穿打底裤女裤韩版薄款百搭夏价格：138,北森服饰专营店,7516,0.1857,268,//detail.tmall.com/item.htm?id=521748590119,//beisenfs.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,2,1,3,5,1,1,3,4,3,1
1,//img.alicdn.com/bao/uploaded/i2/TB1OpR.PpXXXX...,2,打底裤女裤显瘦黑色2017新款韩版百搭春秋小脚不加绒薄款春季外穿价格：55,雅羊人旗舰店,6974,0.0203,171,//detail.tmall.com/item.htm?id=544145512120,//yayangren.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,3,4,4,3,2,3,2,3,2,2
2,//img.alicdn.com/bao/uploaded/i2/TB1YJvGPVXXXX...,3,春款打底裤女外穿薄款2017新款春季学生修身百搭九高腰小脚铅笔裤价格：138,爱购依服饰旗舰店,6461,94.0968,364,//detail.tmall.com/item.htm?id=545938657783,//agoyi.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,,,,404,,,,,,3
3,//img.alicdn.com/bao/uploaded/i2/TB1aanLPVXXXX...,4,春季打底裤女外穿薄款韩版2017新款女裤黑色裤子显瘦百搭潮小脚裤价格：38,385度旗舰店,4945,0.4193,301,//detail.tmall.com/item.htm?id=546377760087,//385du.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,,,,,375,209,124,40,10,4
4,//img.alicdn.com/bao/uploaded/i3/TB1QuiGPFXXXX...,5,光泽裤打底裤女裤外穿薄款踩脚加绒皮裤黑色春季2017新款韩版百搭价格：108,逸莲旗舰店,4741,0.0059,214,//detail.tmall.com/item.htm?id=536646962438,//yilianfushi.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,37,9,26,40,45,41,57,84,6,5
5,//img.alicdn.com/bao/uploaded/i1/TB1fWcNNVXXXX...,6,打底裤女裤春季2017新款女韩版小脚百搭显瘦黑春秋外穿不加绒薄款价格：98,hanchao韩巢旗舰店,4738,0.0530,298,//detail.tmall.com/item.htm?id=521970368186,//hanchao.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,6,3,5,8,5,2,4,5,5,6
6,//img.alicdn.com/bao/uploaded/i3/TB1mJDWPpXXXX...,7,打底裤女裤高腰黑色春季2017新款韩版百搭春秋小脚外穿不加绒薄款价格：98,雅羊人旗舰店,4460,0.0308,286,//detail.tmall.com/item.htm?id=525731921296,//yayangren.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,8,8,6,7,4,5,5,6,8,7
7,//img.alicdn.com/bao/uploaded/i3/318913797/TB2...,8,南极人2017新款春季打底裤女外穿薄款春秋九分黑韩版百搭显瘦小脚价格：99,南极人衣然调调,4018,0.1975,272,//item.taobao.com/item.htm?id=544636633622,//shop116292746.taobao.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,4,2,2,6,3,7,7,2,4,8
8,//img.alicdn.com/bao/uploaded/i2/TB1NpqRPVXXXX...,9,2017新款打底裤外穿显瘦韩版百搭春秋小脚裤黑色薄款春季九分女裤价格：89,唐女旗舰店,3830,0.3071,184,//detail.tmall.com/item.htm?id=546088644389,//tangnv.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,,,96,80,61,38,20,12,15,9
9,//img.alicdn.com/bao/uploaded/i2/TB1AiKFLpXXXX...,10,黑色小脚裤铅笔裤弹力紧身裤裤子女裤薄款仿牛仔打底裤女外穿春季价格：129,jnnhjnnh江南女孩旗舰店,3307,0.0528,195,//detail.tmall.com/item.htm?id=521645721686,//jnnhjnnh.tmall.com,https://sycm.taobao.com/mq/industry/rank/indus...,...,7,10,7,10,7,9,8,7,9,10
